In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
import spm1d.stats.c
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors.v2021 import register_cmaps
import spm1d
from stressgait_analysis.dataset import StressGaitDataset

# Inset axes
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
%matplotlib widget
%load_ext autoreload
%autoreload 2

from matplotlib.patches import Rectangle, ConnectionPatch


import warnings
warnings.simplefilter(action='ignore')
import numpy as np

In [ ]:
plt.close("all")
register_cmaps()
palette = sns.color_palette("faculties_light")
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
#plt.rcParams["font.sans-serif"] = "FAUSans Office"

controls_color = sns.color_palette("wiso")[1]
omc_color = sns.color_palette("med_dark")[1]

stressgait = {
    "OMC": omc_color,
    "Control": controls_color
}

controls_color = sns.color_palette("wiso")[2]
omc_color = sns.color_palette("med_dark")[2]

stressgait2 = {
    "OMC": omc_color,
    "Control": controls_color
}



In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])

export_path = Path("../../exports")
plot_path = export_path.joinpath("plots")
bp.utils.file_handling.mkdirs([export_path, plot_path])

export_path

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True, specify_bouts=True, specify_speed=True)


# Gait ROMs

In [ ]:
kinematics = dataset.kinematics.droplevel(level="bout")

In [ ]:
bad_cycles = kinematics[kinematics['elbow_angle']<80].index.droplevel('percentage_of_stride').drop_duplicates()
# Step 2: Filter out those cycles
kinematics_clean_arms = kinematics[~kinematics.index.droplevel('percentage_of_stride').isin(bad_cycles)]

In [ ]:
def get_rom(trial):
    max = trial.groupby(["cycle_idx"]).max()
    min = trial.groupby(["cycle_idx"]).min()


    return np.mean(max - min, axis=0)


rom = kinematics.groupby(["participant", "condition", "speed"]).apply(get_rom)
rom_per_person = rom.groupby(["participant", "condition"]).mean().loc[:,["elbow_angle", "shoulder_angle", "arm_swing"]]

rom_arms = kinematics_clean_arms.groupby(["participant", "condition", "speed"]).apply(get_rom)
rom_per_person_arms = rom_arms.groupby(["participant", "condition"]).mean()

rom_per_person.loc[:,["elbow_angle", "shoulder_angle", "arm_swing"]] = rom_per_person_arms.loc[:,["elbow_angle", "shoulder_angle", "arm_swing"]]


rom_per_group = rom_per_person.groupby(["condition"]).mean()
rom_per_group

In [ ]:
df_long = rom_per_person.reset_index().melt(
    id_vars=["participant", "condition"],  # Keep these columns
    var_name="angle",  # Name for the melted variable column
    value_name="value"  # Name for the melted value column
)
plt.figure()
sns.boxplot(x="angle", y="value", hue="condition", data=df_long, showfliers=False)

In [ ]:
import matplotlib.patches as mpatches
sns.set_style("white")
fig, axs=plt.subplots(figsize=(11,5),ncols=3, gridspec_kw={'wspace': 0.7})

for c, ax in zip(rom_per_person.columns, axs):
    sns.boxplot(data = rom_per_person[c].reset_index(), x="condition", y=c, hue = "condition", ax=ax, showfliers=True, palette=stressgait)
    sns.swarmplot(data = rom_per_person[c].reset_index(), x="condition", y=c, hue = "condition", ax=ax, palette=stressgait2, size=8)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)



legend_elements = [
   
    mpatches.Patch(facecolor=stressgait['Control'], label='Control'),
     mpatches.Patch(facecolor=stressgait['OMC'], label='OMC'),
]


axs[0].set_ylabel("Elbow ROM [°]", fontsize=16)
axs[0].tick_params(axis='y', colors='black', labelsize=14, left=True, labelbottom=True)

axs[1].set_ylabel("Shoulder ROM [°]", fontsize=16)
axs[2].set_ylabel("Arm swing ROM [pixel]", fontsize=16)
axs[0].set_xlabel("")
axs[1].set_xlabel("")
axs[2].set_xlabel("")
axs[0].set_xticklabels(['Control','OMC'])
axs[1].set_xticklabels(['Control', 'OMC'])
axs[2].set_xticklabels(['Control','OMC'])
axs[1].tick_params(axis='y', colors='black', labelsize=14, left=True, labelbottom=True)
axs[2].tick_params(axis='y', colors='black', labelsize=14, left=True, labelbottom=True)
axs[0].tick_params(axis='x', labelsize=16)
axs[1].tick_params(axis='x', labelsize=16)
axs[2].tick_params(axis='x', labelsize=16)

plt.tight_layout()


plt.savefig("../../exports/plots/ROM.pdf")

In [ ]:
normality_results = df_long.groupby(['condition', "angle"], group_keys=False)["value"].apply(lambda g: pg.normality(g))
normality_results

In [ ]:
test_results = df_long.groupby(["angle"], group_keys=False).apply(lambda g: pg.pairwise_tests(g, parametric=True, dv = "value", between="condition"))
test_results["p_corr"] = test_results["p-unc"]*len(df_long.angle.unique())
test_results["p_corr"].clip(upper=1, inplace=True)
test_results